<img src="asset/img1.jpg" width=150 height=200 align="right">
<h1 style="color:purple; font-family:Helvetica Neue">EMAIL SPAM FILTER</h1>
<h3 style="color:blue; font-family:Helvetica Neue">Training</h3>
<hr>
<h7>Source🤖:<a href="https://github.com/APTUUU/Email-Spam-Detector">Email Spam Detector Repository</a></h7>

## Imports:

In [1]:
import pandas as pd
import numpy as np

## Constants:

In [2]:
PROB_FILE = 'SpamData/03_Testing/token-prob.csv'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'
TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-feature.txt'

DATA_JSON_FILE = 'SpamData/01_Processing/email-text-data.json'
WORD_ID_FILE = 'SpamData/01_Processing/word-by-id.csv'
TRAINING_DATA_FILE  = 'SpamData/02_Training/tain-data.txt'
TESTING_DATA_FILE  = 'SpamData/02_Training/test-data.txt'


VOCAB_SIZE = 2501

## Read and Load Features:

In [3]:
sparse_train_data = np.loadtxt(fname=TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(fname=TESTING_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_test_data.shape

(106928, 4)

In [6]:
sparse_train_data.shape

(241405, 4)

### How to create an empty DF:

In [7]:
column_names = ['DOC_ID'] + ['CLASSIFICATION'] + list(range(0,VOCAB_SIZE))

In [8]:
index_names = np.unique(sparse_train_data[ :,0])

In [9]:
full_train_data = pd.DataFrame(columns=column_names, index=index_names)
full_train_data.fillna(value=0, inplace=True)

In [10]:
full_train_data

,DOC_ID,CLASSIFICATION,0,1,2,3,4,5,6,7,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create Full Matrix from Sparse Matrix:

In [11]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Converts a sparse matrix into a full matrix in a pandas DataFrame format.

    Parameters:
        sparse_matrix (numpy.ndarray): The sparse matrix to be converted.
        nr_words (int): The number of words.
        doc_idx (int, optional): Index for the document number in the sparse matrix (default: 0).
        word_idx (int, optional): Index for the word ID in the sparse matrix (default: 1).
        cat_idx (int, optional): Index for the classification in the sparse matrix (default: 2).
        freq_idx (int, optional): Index for the frequency in the sparse matrix (default: 3).

    Returns:
        pandas.DataFrame: A DataFrame containing the full representation of the sparse data.
    """
    
    column_names = ['DOC_ID'] + ['CLASSIFICATION'] + list(range(0,nr_words))
    index_names = np.unique(sparse_matrix[ :,0])
    full_matrix = pd.DataFrame(columns=column_names, index=index_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        
        doc_nr = sparse_matrix[i, doc_idx]
        word_id = sparse_matrix[i, word_idx]
        label = sparse_matrix[i, cat_idx]
        occurence = sparse_matrix[i, freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CLASSIFICATION'] = label
        full_matrix.at[doc_nr, word_id] = occurence
     
    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix

In [12]:
%%time
full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

Wall time: 18.8 s


In [13]:
full_train_data

,CLASSIFICATION,0,1,2,3,4,5,6,7,8,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,1,0,2,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,2,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,2,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,2,1,0,1,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0
4,1,3,0,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,1,1,0,1,0,2,0,1,3,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0


## Training Naive Bayes Model:

### Calculating the Probability of Spam:

In [14]:
full_train_data.CLASSIFICATION.value_counts()

1    2705
0    1309
Name: CLASSIFICATION, dtype: int64

In [15]:
prob_spam = full_train_data.CLASSIFICATION.value_counts().values[1] / full_train_data.CLASSIFICATION.count()
prob_spam

0.3261086198305929

### Total Number of Words / Tokens :

In [16]:
full_train_features = full_train_data.drop("CLASSIFICATION", axis=1)
total_number_words = full_train_features.sum(1).sum()

In [17]:
total_number_words

380760

### Toatal Number of Words in each Category:

In [18]:
spam_train_data = full_train_data.query('CLASSIFICATION == 0')
total_number_words_spam = spam_train_data.sum(1).sum()
total_number_words_spam

170417

In [19]:
ham_train_data = full_train_data.query('CLASSIFICATION == 1')
total_number_words_ham = ham_train_data.sum(1).sum()
total_number_words_ham

213048

In [20]:
spam_train_data.sum(1).mean()

130.18869365928188

In [21]:
ham_train_data.sum(1).mean()

78.76081330868762

In [22]:
words_frequencies = pd.DataFrame(index=list(range(0,VOCAB_SIZE)), columns=['IN_SPAM', 'IN_HAM', 'OVERALL'])
words_frequencies.index.name = 'WORD_ID'

In [23]:
words_frequencies.IN_HAM = ham_train_data.sum(0)[1:].values + 1
words_frequencies.IN_SPAM = spam_train_data.sum(0)[1:].values + 1
words_frequencies.OVERALL = ham_train_data.sum(0)[1:].values + spam_train_data.sum(0)[1:].values + 2

## P(Token | Spam) / P(Token | Ham) / P(Token | Overall) - Probability that a token occurs given that the email is :  spam / ham / overall  : 

In [24]:
words_frequencies['PROB_SPAM'] = words_frequencies.IN_SPAM / (total_number_words_spam + VOCAB_SIZE)
words_frequencies['PROB_HAM'] =  words_frequencies.IN_HAM / (total_number_words_ham + VOCAB_SIZE)
words_frequencies['PROB_OVERALL'] = words_frequencies.OVERALL / (total_number_words + 2*VOCAB_SIZE)

In [25]:
words_frequencies

,IN_SPAM,IN_HAM,OVERALL,PROB_SPAM,PROB_HAM,PROB_OVERALL
WORD_ID,,,,,,
0,1631,4622,6253,0.009432,0.021443,0.016209
1,1054,1830,2884,0.006095,0.008490,0.007476
2,1795,850,2645,0.010381,0.003943,0.006857
3,1073,1564,2637,0.006205,0.007256,0.006836
4,1218,1222,2440,0.007044,0.005669,0.006325
...,...,...,...,...,...,...
2496,36,2,38,0.000208,0.000009,0.000099
2497,1,18,19,0.000006,0.000084,0.000049
2498,1,37,38,0.000006,0.000172,0.000099


## Save Trained Data :

In [26]:
words_frequencies.to_csv(PROB_FILE)

In [27]:
%%time
full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

Wall time: 9.47 s


In [35]:
# full_test_data.set_index("DOC_ID", inplace=True)

In [30]:
full_test_data

,CLASSIFICATION,0,1,2,3,4,5,6,7,8,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,1,2,0,1,2,1,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
12,1,1,1,0,0,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
14,1,2,3,0,2,0,1,3,1,0,...,0,0,0,0,0,0,0,0,0,0
15,1,1,1,0,0,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
17,1,2,2,0,1,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,0,0,2,2,1,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
5786,0,0,0,0,0,2,3,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5788,0,0,0,1,2,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [31]:
y_test = full_test_data.CLASSIFICATION

In [32]:
x_test = full_test_data.drop('CLASSIFICATION', axis=1)

In [33]:
x_test

,0,1,2,3,4,5,6,7,8,9,...,2491,2492,2493,2494,2495,2496,2497,2498,2499,2500
DOC_ID,,,,,,,,,,,,,,,,,,,,,
8,2,0,1,2,1,1,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
12,1,1,0,0,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
14,2,3,0,2,0,1,3,1,0,3,...,0,0,0,0,0,0,0,0,0,0
15,1,1,0,0,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17,2,2,0,1,1,0,1,0,0,2,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5783,0,2,2,1,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
5786,0,0,0,0,2,3,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5788,0,0,1,2,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [34]:
np.savetxt(fname=TEST_TARGET_FILE, X=y_test, fmt='%d')
np.savetxt(fname=TEST_FEATURE_MATRIX, X=x_test, fmt='%d')